In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
# tup
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  
    # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() 
    # default distribution strategy in Tensorflow. Works on CPU and single GPU.
print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.52.55.194:8470
INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Initializing the TPU system: grpc://10.52.55.194:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.52.55.194:8470


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS:  8


In [3]:
# fashion dataset load
fashion_mnist = keras.datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

4431872/4422102 [==============================] - 0s 0us/step


In [4]:
# dataset shape
train_images.shape, test_images.shape

((60000, 28, 28), (10000, 28, 28))

In [5]:
# the number of classes
num_class = len(np.unique(test_labels))

In [6]:
# make one-hot label
train_labels = keras.utils.to_categorical(train_labels)
test_labels = keras.utils.to_categorical(test_labels)

In [19]:
# preprocess function
# normalize x
def preprocess(x, y):
  x = x/255
  x = tf.expand_dims(x, 3)
  return x, y

In [20]:
# make trainset
batch_size = 64
train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.map(preprocess).shuffle(1000, seed=42)    # shuffle

test_dataset  = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.map(preprocess)

In [21]:
with strategy.scope():
  # model
  model = keras.models.Sequential([
              keras.layers.Conv2D(8, 5, activation='relu', input_shape=(28, 28, 1)),
              keras.layers.MaxPool2D(pool_size=(3, 3)),
              keras.layers.Conv2D(16, 3, activation='relu'),
              keras.layers.MaxPool2D(pool_size=(2, 2)),
              keras.layers.Flatten(),
              keras.layers.Dense(20, activation='relu'),
              keras.layers.Dense(num_class, activation='softmax')
  ])

  model.compile(
      optimizer='adam',
      loss='categorical_crossentropy',
      metrics='accuracy'
  )

In [22]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 24, 24, 8)         208       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 8, 8, 8)           0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 6, 6, 16)          1168      
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 3, 3, 16)          0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 144)               0         
_________________________________________________________________
dense_10 (Dense)             (None, 20)                2900      
_________________________________________________________________
dense_11 (Dense)             (None, 10)               

In [23]:
hist = model.fit(train_dataset, validation_data=test_dataset, epochs=10)

Epoch 1/10
938/938 [==============================] - 19s 16ms/step - loss: 0.7801 - accuracy: 0.7191 - val_loss: 0.5717 - val_accuracy: 0.7891
Epoch 2/10
938/938 [==============================] - 11s 12ms/step - loss: 0.5025 - accuracy: 0.8159 - val_loss: 0.4885 - val_accuracy: 0.8192
Epoch 3/10
938/938 [==============================] - 12s 12ms/step - loss: 0.4389 - accuracy: 0.8408 - val_loss: 0.4492 - val_accuracy: 0.8410
Epoch 4/10
938/938 [==============================] - 12s 13ms/step - loss: 0.4050 - accuracy: 0.8541 - val_loss: 0.4185 - val_accuracy: 0.8491
Epoch 5/10
938/938 [==============================] - 12s 13ms/step - loss: 0.3806 - accuracy: 0.8620 - val_loss: 0.3911 - val_accuracy: 0.8644
Epoch 6/10
938/938 [==============================] - 12s 13ms/step - loss: 0.3639 - accuracy: 0.8684 - val_loss: 0.3798 - val_accuracy: 0.8647
Epoch 7/10
938/938 [==============================] - 12s 13ms/step - loss: 0.3517 - accuracy: 0.8726 - val_loss: 0.3966 - val_accuracy: